In [ ]:
## vote choice as driven by *housing tenure* x *age* x *education*

In [1]:
# %%time
## Just an easy way to load utility functions
dataset_name = "W19_only"
df_list = []

%matplotlib inline 
%run f2f_header.py {dataset_name} {df_list}


Toggle code

In [2]:
global BES_label_list, BES_df_list

# %run f2f_utility.py

BES_df_list = pickle.load( open( "E:\\BES_analysis_data\\Face_to_face\\BES_df_list.pickle", "rb" ) )
f2f_harmonised = pd.read_pickle("E:\\BES_analysis_data\\Face_to_face\\f2f_harmonised.zip", compression='zip')
BES_label_list = pickle.load( open( "E:\\BES_analysis_data\\Face_to_face\\BES_label_list.pickle", "rb" ) )
demo_var_dict = pickle.load( open( "E:\\BES_analysis_data\\Face_to_face\\demo_var_dict.pickle", "rb" ) )
var_type_dict_nonans = pickle.load( open( "E:\\BES_analysis_data\\Face_to_face\\var_type_dict_nonans.pickle", "rb" ) )

generic_cols = f2f_harmonised.columns
list_of_scale_harm_vars = ["Age","year_past_min_sch_leave_age","in_school_past_min_age"]



## BUGS

BES_label_list["1959"]["v1236"] = 'LOCAL ELECTION VOTE 1963(2nd)'
BES_label_list["1964"]["v1236"] = 'LOCAL ELECTION VOTE 1963(2nd)'
BES_label_list["1966"]["v1236"] = 'LOCAL ELECTION VOTE 1963(2nd)'
BES_label_list["1970"]["v1236"] = 'LOCAL ELECTION VOTE 1963(2nd)'

BES_label_list["2015"]['edlevel'] = 'Education level (summary)'



In [ ]:
# https://twitter.com/chriscurtis94/status/1518576882756825090

# It isn't. My point wasn't that housing tenure drives VI, although I think it does, more that house prices going up will disproportionately help Tory voters. 

# I'm sure someone else has run it with age and education controls.

# 2019GE Vote choice

In [22]:
import statsmodels.api as sm

In [ ]:
    import statsmodels.api as sm
    Y = df_simp[mask][var_stub]
    ## default mean imputation!
    X = df_simp[mask][specific_vars].astype('float').fillna(df_simp[mask][specific_vars].mean())
#     X = sm.add_constant(X)
    X["const"]=1.0
    model = sm.OLS(Y,X)
    results = model.fit()
    Xnew = df_simp[train_columns].astype('float')
#     Xnew = sm.add_constant(Xnew)
    Xnew["const"]=1.0
    statsmodels_pred = model.predict(results.params,Xnew)
    display( results.summary() );

In [ ]:
## get variables

In [17]:
f2f_harmonised["bloc_complete"]

index
0        Neither
1        Neither
2       Left-Lib
3        Neither
4        Neither
          ...   
2917         NaN
2918         NaN
2919         NaN
2920    Left-Lib
2921         NaN
Name: bloc_complete, Length: 52731, dtype: object

In [40]:
df = f2f_harmonised[f2f_harmonised["dataset"]=="2019"][["housing_det","age","vote_complete","bloc_complete","wt"]].copy()
df["edlevel"] = BES_df_list["2019"]["edlevel"]

In [30]:
df["edlevel"].value_counts()

Undergraduate        1249
No qualifications     762
A-level               681
GCSE                  451
Postgrad              423
Below GCSE            202
Name: edlevel, dtype: int64

In [31]:
df["housing_det"].value_counts()

OWN        1437
MORT       1256
RENT-PR     550
RENT-LA     331
RENT-HA     291
Name: housing_det, dtype: int64

In [41]:
mask = df[["edlevel","age","housing_det","vote_complete","wt"]].notnull().all(axis=1)
mask = mask&(df["vote_complete"].isin(["CON","LAB"]))

X = pd.get_dummies(df[["edlevel","age","housing_det"]]).drop(["housing_det_OWN","edlevel_Undergraduate"],axis=1)[mask]
Y = df["vote_complete"][mask]=="CON"

X["const"]=1.0
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          vote_complete   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     30.07
Date:                Sun, 01 May 2022   Prob (F-statistic):           1.59e-54
Time:                        23:17:53   Log-Likelihood:                -1305.2
No. Observations:                2051   AIC:                             2632.
Df Residuals:                    2040   BIC:                             2694.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
age                           0.0072      0.001      9.883      0.000       0.006       0.009
edlevel_No qualifications     0.0547      0.031      1.759      0.079      -0.006       0.116
edlevel_Below GCSE            0.1436      0.048      2.995      0.003       0.050       0.238
edlevel_GCSE                  0.1241      0.033      3.706      0.000       0.058       0.190
edlevel_A-level               0.1099      0.030      3.698      0.000       0.052       0.168
edlevel_Postgrad             -0.1170      0.035     -3.313      0.001      -0.186      -0.048
housing_det_MORT             -0.0007      0.028     -0.027      0.979      -0.055       0.054
housing_det_RENT-HA          -0.1142      0.044     -2.598      0.009      -0.200      -0.028
housing_det_RENT-LA          -0.2436      0.044     -5.484      0.000      -0.331      -0.157
housing_det_RENT-PR          -0.1196      0.038     -3.178      0.002      -0.193      -0.046
const                         0.2048      0.050      4.063      0.000       0.106       0.304
==============================================================================
Omnibus:                     7946.824   Durbin-Watson:                   1.769
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              192.407
Skew:                          -0.362   Prob(JB):                     1.66e-42
Kurtosis:                       1.686   Cond. No.                         367.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
mask = df[["edlevel","age","housing_det","vote_complete","wt"]].notnull().all(axis=1)
mask = mask&(df["vote_complete"].isin(["CON","LAB"]))

X = pd.get_dummies(df[["edlevel","age","housing_det"]]).drop(["housing_det_OWN","edlevel_Undergraduate"],axis=1)[mask]
Y = df["vote_complete"][mask]=="CON"
Y = Y.astype('int')

import statsmodels.formula.api as smf

# X["const"]=1.0
XY = pd.concat([X,Y],axis=1)
XY = XY.rename(columns = {"edlevel_No qualifications":"edlevel_No_qualifications",'edlevel_Below GCSE':'edlevel_Below_GCSE',
                         "housing_det_RENT-HA":"housing_det_RENT_HA","housing_det_RENT-LA":"housing_det_RENT_LA","housing_det_RENT-PR":"housing_det_RENT_PR",
                         "edlevel_A-level":"edlevel_A_level"})
# 'age', 'edlevel_No qualifications', 'edlevel_Below GCSE',
#        'edlevel_GCSE', 'edlevel_A-level', 'edlevel_Postgrad',
#        'housing_det_MORT', 'housing_det_RENT-HA', 'housing_det_RENT-LA',
#        'housing_det_RENT-PR', 'const', 'bloc_complete'

fit = smf.glm("vote_complete ~ age + edlevel_No_qualifications + edlevel_Below_GCSE + edlevel_GCSE + edlevel_A_level + edlevel_Postgrad +"+\
              "housing_det_MORT + housing_det_RENT_HA + housing_det_RENT_LA + housing_det_RENT_PR",
              family=sm.families.Binomial(),data=XY,freq_weights=df["wt"][mask]).fit()
fit.summary()

# model = sm.OLS(Y,X)
# results = model.fit(freq_weights=df["wt"][mask])
# results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          vote_complete   No. Observations:                 2051
Model:                            GLM   Df Residuals:                  1712.49
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1058.6
Date:                Sun, 01 May 2022   Deviance:                       2117.2
Time:                        23:44:07   Pearson chi2:                 1.74e+03
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1089
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.4042      0.247     -5.690      0.000      -1.888      -0.921
age                           0.0324      0.004      8.950      0.000       0.025       0.039
edlevel_No_qualifications     0.2179      0.162      1.348      0.178      -0.099       0.535
edlevel_Below_GCSE            0.8585      0.272      3.158      0.002       0.326       1.391
edlevel_GCSE                  0.5245      0.173      3.030      0.002       0.185       0.864
edlevel_A_level               0.5576      0.149      3.736      0.000       0.265       0.850
edlevel_Postgrad             -0.5275      0.181     -2.915      0.004      -0.882      -0.173
housing_det_MORT             -0.0469      0.138     -0.340      0.734      -0.317       0.224
housing_det_RENT_HA          -0.6416      0.235     -2.733      0.006      -1.102      -0.181
housing_det_RENT_LA          -1.1005      0.238     -4.617      0.000      -1.568      -0.633
housing_det_RENT_PR          -0.5309      0.193     -2.747      0.006      -0.910      -0.152
=============================================================================================
"""

In [39]:
mask = df[["edlevel","age","housing_det","bloc_complete"]].notnull().all(axis=1)
mask = mask&(df["bloc_complete"].isin(["Right-Auth","Left-Lib"]))

X = pd.get_dummies(df[["edlevel","age","housing_det"]]).drop(["housing_det_OWN","edlevel_Undergraduate"],axis=1)[mask]
Y = df["bloc_complete"][mask]=="Right-Auth"

X["const"]=1.0
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          bloc_complete   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     29.23
Date:                Sun, 01 May 2022   Prob (F-statistic):           4.57e-54
Time:                        23:11:56   Log-Likelihood:                -1968.8
No. Observations:                2910   AIC:                             3960.
Df Residuals:                    2899   BIC:                             4025.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
age                           0.0060      0.001      9.223      0.000       0.005       0.007
edlevel_No qualifications     0.1029      0.028      3.729      0.000       0.049       0.157
edlevel_Below GCSE            0.1383      0.042      3.273      0.001       0.055       0.221
edlevel_GCSE                  0.1480      0.030      4.976      0.000       0.090       0.206
edlevel_A-level               0.1348      0.026      5.137      0.000       0.083       0.186
edlevel_Postgrad             -0.1046      0.029     -3.600      0.000      -0.162      -0.048
housing_det_MORT              0.0107      0.024      0.443      0.658      -0.037       0.058
housing_det_RENT-HA          -0.0886      0.040     -2.228      0.026      -0.167      -0.011
housing_det_RENT-LA          -0.1608      0.039     -4.089      0.000      -0.238      -0.084
housing_det_RENT-PR          -0.0903      0.032     -2.781      0.005      -0.154      -0.027
const                         0.1228      0.045      2.757      0.006       0.035       0.210
==============================================================================
Omnibus:                    14377.039   Durbin-Watson:                   1.859
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              322.178
Skew:                           0.068   Prob(JB):                     1.10e-70
Kurtosis:                       1.376   Cond. No.                         367.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
mask = df[["edlevel","age","housing_det","bloc_complete"]].notnull().all(axis=1)
mask = mask&(df["bloc_complete"].isin(["Right-Auth","Left-Lib"]))

X = pd.get_dummies(df[["edlevel","age","housing_det"]]).drop(["housing_det_OWN","edlevel_Undergraduate"],axis=1)[mask]
Y = df["bloc_complete"][mask]=="Right-Auth"
Y = Y.astype('int')

import statsmodels.formula.api as smf

# X["const"]=1.0
XY = pd.concat([X,Y],axis=1)
XY = XY.rename(columns = {"edlevel_No qualifications":"edlevel_No_qualifications",'edlevel_Below GCSE':'edlevel_Below_GCSE',
                         "housing_det_RENT-HA":"housing_det_RENT_HA","housing_det_RENT-LA":"housing_det_RENT_LA","housing_det_RENT-PR":"housing_det_RENT_PR",
                         "edlevel_A-level":"edlevel_A_level"})
# 'age', 'edlevel_No qualifications', 'edlevel_Below GCSE',
#        'edlevel_GCSE', 'edlevel_A-level', 'edlevel_Postgrad',
#        'housing_det_MORT', 'housing_det_RENT-HA', 'housing_det_RENT-LA',
#        'housing_det_RENT-PR', 'const', 'bloc_complete'

fit = smf.glm("bloc_complete ~ age + edlevel_No_qualifications + edlevel_Below_GCSE + edlevel_GCSE + edlevel_A_level + edlevel_Postgrad +"+\
              "housing_det_MORT + housing_det_RENT_HA + housing_det_RENT_LA + housing_det_RENT_PR",
              family=sm.families.Binomial(),data=XY,freq_weights=df["wt"][mask]).fit()
fit.summary()

# model = sm.OLS(Y,X)
# results = model.fit(freq_weights=df["wt"][mask])
# results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          bloc_complete   No. Observations:                 2660
Model:                            GLM   Df Residuals:                  2173.46
Model Family:                Binomial   Df Model:                           10
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1397.5
Date:                Sun, 01 May 2022   Deviance:                       2794.9
Time:                        23:37:41   Pearson chi2:                 2.19e+03
No. Iterations:                     4   Pseudo R-squ. (CS):            0.08166
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -1.6107      0.218     -7.384      0.000      -2.038      -1.183
age                           0.0261      0.003      8.276      0.000       0.020       0.032
edlevel_No_qualifications     0.4451      0.141      3.151      0.002       0.168       0.722
edlevel_Below_GCSE            0.6036      0.209      2.890      0.004       0.194       1.013
edlevel_GCSE                  0.5964      0.147      4.062      0.000       0.309       0.884
edlevel_A_level               0.5837      0.130      4.502      0.000       0.330       0.838
edlevel_Postgrad             -0.5345      0.162     -3.304      0.001      -0.852      -0.217
housing_det_MORT              0.0025      0.119      0.021      0.983      -0.230       0.235
housing_det_RENT_HA          -0.4178      0.208     -2.013      0.044      -0.825      -0.011
housing_det_RENT_LA          -0.7889      0.214     -3.690      0.000      -1.208      -0.370
housing_det_RENT_PR          -0.4658      0.172     -2.702      0.007      -0.804      -0.128
=============================================================================================
"""

In [57]:
XY

,age,edlevel_No_qualifications,edlevel_Below_GCSE,edlevel_GCSE,edlevel_A-level,edlevel_Postgrad,housing_det_MORT,housing_det_RENT-HA,housing_det_RENT-LA,housing_det_RENT-PR,const,bloc_complete
index,,,,,,,,,,,,
2,33.0,0,0,0,0,1,1,0,0,0,1.0,False
5,53.0,0,0,1,0,0,0,0,0,1,1.0,True
6,27.0,0,0,0,0,0,1,0,0,0,1.0,True
7,62.0,1,0,0,0,0,0,0,1,0,1.0,True
8,63.0,0,0,0,0,0,0,0,0,0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3939,60.0,0,0,0,0,0,0,0,0,0,1.0,True
3942,46.0,0,0,1,0,0,1,0,0,0,1.0,True
3943,56.0,0,0,0,0,1,1,0,0,0,1.0,True


In [36]:
df["bloc_complete"].value_counts()

Left-Lib      1616
Right-Auth    1486
Neither        823
Name: bloc_complete, dtype: int64

In [11]:
BES_df_list["2019"]["education"].value_counts()

First degree                                             703
No qualification                                         679
Postgraduate degree                                      423
GCSE A*-C, CSE grade 1, O level grade A-C                322
HNC/HND, City&Guilds level 4, NVQ/SVQ 4/5                309
A level or equivalent                                    300
ONC/OND, City&Guilds level 3, NVQ/SVQ 3                  189
City&Guilds level 2, NVQ/SVQ 2 and equiv                 158
GCSE D-G, CSE grades 2-5, O level D-E                    105
University/Polytechnic diploma                           101
Clerical and commercial qualifications                    95
Prefer not to say                                         87
None of these                                             83
Teaching qualification                                    80
Other technical, professional or higher qualification     78
Recognised trade apprenticeship                           67
Nursing qualification   

In [13]:

BES_df_list["2019"]["edlevel"].value_counts()

Undergraduate        1249
No qualifications     762
A-level               681
GCSE                  451
Postgrad              423
Below GCSE            202
Name: edlevel, dtype: int64

In [19]:
BES_df_list["2019"]["edlevel"]

0           Undergraduate
1                     NaN
2                Postgrad
3       No qualifications
4              Below GCSE
              ...        
3941             Postgrad
3942                 GCSE
3943             Postgrad
3944           Below GCSE
3945        Undergraduate
Name: edlevel, Length: 3946, dtype: category
Categories (6, object): ['No qualifications' < 'Below GCSE' < 'GCSE' < 'A-level' < 'Undergraduate' < 'Postgrad']

In [12]:
BES_label_list["2019"]

{'finalserialno': 'Final Serial Number',
 'agency': 'Fieldwork agency',
 'Y10A': 'Including yourself, how many adults aged 18 and over currently live in your hous',
 'Y10B1': 'A British Citizen',
 'Y10B2': 'A Citizen of the Republic of Ireland',
 'Y10B3': 'A Citizen of a Commonwealth country who has a ?right to remain? in the UK',
 'Y10B4': 'A Citizen of an EU country',
 'Y10B5': 'None of these',
 'a01': 'Most important issue (open text)',
 'a01_code': 'Most important issue (coded)',
 'a02': 'Best party on most important issue',
 'a03': 'Interest in politics',
 'm02_1': "Politicians don't care what people like me think",
 'm02_2': 'It is often difficult for me to understand what is going on',
 'm02_3': 'People like me have no say in what government does',
 'm02_4': 'The main political parties in Britain are pretty much all the same',
 'm02_5': 'UK parties/politicians fight eachother rather than further public interest',
 'm02_6': 'Politicians ignore the issues I really care about',
 'b